In [4]:
import pandas as pd
variants = pd.read_csv('out/variants.vcf.gz', compression='gzip', comment='#', sep='\t', header=None)
variants.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,Lu1,10,.,G,A,33.41630,.,VDB=0.06;SGB=-0.453602;RPBZ=-1.22474;MQBZ=1.22...,GT:PL,./.:.,...,"1/1:39,9,0",./.:.,"1/1:40,9,0",./.:.,./.:.,"0/1:37,0,52",./.:.,"1/1:54,8,0",./.:.,./.:.
1,Lu1,12,.,T,A,5.68822,.,SGB=-0.379885;RPBZ=-1;MQBZ=1;MQSBZ=-1;BQBZ=-1;...,GT:PL,./.:.,...,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.
2,Lu1,15,.,G,A,54.41460,.,VDB=0.06;SGB=-0.453602;MQSBZ=-1;MQ0F=0.5;MQ=20...,GT:PL,./.:.,...,"1/1:39,9,0",./.:.,"1/1:44,9,0",./.:.,"1/1:53,12,0","0/1:40,0,45","1/1:35,6,0","1/1:72,15,0",./.:.,"1/1:41,6,0"
3,Lu1,19,.,T,A,4.97970,.,SGB=-0.379885;RPBZ=-1;MQBZ=1;MQSBZ=-1;BQBZ=1;S...,GT:PL,./.:.,...,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.
4,Lu1,33,.,G,T,34.41590,.,VDB=0.160332;SGB=-0.511536;MQ0F=0;MQ=21;RPBZ=-...,GT:PL,./.:.,...,"1/1:42,12,0","1/1:57,9,0","1/1:43,9,0",./.:.,./.:.,"0/1:40,0,45","1/1:50,3,0","0/1:56,0,22",./.:.,./.:.


In [11]:
# Rename columns to match GFF file specs
col_names = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT'] + [i for i in range(1,62)]
variants.rename(columns={i:col_names[i] for i in range(0,69)}, inplace=True)
variants.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,1,...,52,53,54,55,56,57,58,59,60,69
0,Lu1,10,.,G,A,33.41630,.,VDB=0.06;SGB=-0.453602;RPBZ=-1.22474;MQBZ=1.22...,GT:PL,./.:.,...,"1/1:39,9,0",./.:.,"1/1:40,9,0",./.:.,./.:.,"0/1:37,0,52",./.:.,"1/1:54,8,0",./.:.,./.:.
1,Lu1,12,.,T,A,5.68822,.,SGB=-0.379885;RPBZ=-1;MQBZ=1;MQSBZ=-1;BQBZ=-1;...,GT:PL,./.:.,...,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.
2,Lu1,15,.,G,A,54.41460,.,VDB=0.06;SGB=-0.453602;MQSBZ=-1;MQ0F=0.5;MQ=20...,GT:PL,./.:.,...,"1/1:39,9,0",./.:.,"1/1:44,9,0",./.:.,"1/1:53,12,0","0/1:40,0,45","1/1:35,6,0","1/1:72,15,0",./.:.,"1/1:41,6,0"
3,Lu1,19,.,T,A,4.97970,.,SGB=-0.379885;RPBZ=-1;MQBZ=1;MQSBZ=-1;BQBZ=1;S...,GT:PL,./.:.,...,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.,./.:.
4,Lu1,33,.,G,T,34.41590,.,VDB=0.160332;SGB=-0.511536;MQ0F=0;MQ=21;RPBZ=-...,GT:PL,./.:.,...,"1/1:42,12,0","1/1:57,9,0","1/1:43,9,0",./.:.,./.:.,"0/1:40,0,45","1/1:50,3,0","0/1:56,0,22",./.:.,./.:.
